In [140]:
# pip install twilio

# import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID, TWILIO_AUTH_TOKEN, PHONE_NUMBER, API_KEY_WAPI
# import time

# from requests import Request,Session
# from requests.exceptions import ConnectionError,Timeout,TooManyRedirects
import json

import pandas as pd
import requests
# from bs4 import BeautifulSoup
from tqdm import tqdm
from time import sleep

# from datetime import datetime

In [120]:
query = "San Salvador de Jujuy"
api_key = API_KEY_WAPI

url_clima = f"http://api.weatherapi.com/v1/forecast.json?key={api_key}&q={query}&days=1&aqi=no&alerts=no"

response = requests.get(url_clima).json()
# Ver la estructura que almacena la variable response en: 'estructura.json'

In [121]:
# Llaves principales de la estructura del JSON. 
print(response.keys())

dict_keys(['location', 'current', 'forecast'])


In [122]:
# Llaves principales de la estructura de segundo nivel dentro de "forecast"
print(response['forecast']['forecastday'][0].keys())

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])


In [123]:
# Dentro de la llave 'hour' tenemos una lista de diccionarios correspondientes al pronostico en cada hora del dia, es decir 24 diccionarios 
print(len(response['forecast']['forecastday'][0]['hour']))

24


In [124]:
# TOMANDO COMO EJEMPLO EL DECIMO DICCIONARIO (10:00hs)
# Obtener fecha
print(response['forecast']['forecastday'][0]['hour'][10]['time'].split()[0])
# Obtener hora
print(response['forecast']['forecastday'][0]['hour'][10]['time'].split()[1].split(':')[0])
# Obtener condicion
print(response['forecast']['forecastday'][0]['hour'][10]['condition']['text'])
# Obtener temperatura
print(response['forecast']['forecastday'][0]['hour'][10]['temp_c'])
# Obtener respuesta a llovera? 1=si 0=no
print(response['forecast']['forecastday'][0]['hour'][10]['will_it_rain'])
# Obtener probabilidad de lluvia
print(response['forecast']['forecastday'][0]['hour'][10]['chance_of_rain'])

2023-11-21
10
Sunny
28.0
0
0


In [125]:
def get_forecast(response,i):
    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = int(response['forecast']['forecastday'][0]['hour'][i]['time'].split()[
        1].split(':')[0])
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temperatura = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    lluvia = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    prob_lluvia = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    return fecha,hora,condicion,temperatura,lluvia,prob_lluvia

In [126]:
datos = []
for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour='green'):    
    datos.append(get_forecast(response,i))    
    #sleep(0.5)    

100%|██████████| 24/24 [00:00<?, ?it/s]


In [127]:
col = ['Fecha','Hora','Condicion','Temperatura','Lluvia','Prob_lluvia']
df = pd.DataFrame(datos,columns=col)

In [128]:
df

,Fecha,Hora,Condicion,Temperatura,Lluvia,Prob_lluvia
0,2023-11-21,0,Clear,19.1,0,0
1,2023-11-21,1,Clear,30.0,0,0
2,2023-11-21,2,Clear,17.4,0,0
3,2023-11-21,3,Clear,17.2,0,0
4,2023-11-21,4,Patchy rain possible,16.0,1,71
5,2023-11-21,5,Clear,15.9,0,0
6,2023-11-21,6,Clear,15.7,0,0
7,2023-11-21,7,Sunny,17.9,0,0
8,2023-11-21,8,Sunny,23.9,0,0
9,2023-11-21,9,Sunny,25.9,0,0


In [130]:
df.dtypes

Fecha           object
Hora             int64
Condicion       object
Temperatura    float64
Lluvia           int64
Prob_lluvia      int64
dtype: object

In [133]:
df_rain = df[(df['Lluvia']==1) & (df['Hora']>=6)]
df_rain = df_rain[['Hora','Condicion']]
df_rain = df_rain.set_index('Hora')
df_rain

,Condicion
Hora,
11,Patchy rain possible
12,Patchy rain possible
14,Patchy rain possible
16,Patchy light rain with thunder
17,Patchy light rain with thunder


In [137]:
# MENSAJE
mensaje = f"Hola!\nEl pronostico de lluvia para hoy {df['Fecha'][0]} en {query} es:\n{str(df_rain)}"
mensaje

'Hola!\nEl pronostico de lluvia para hoy 2023-11-21 en San Salvador de Jujuy es:\n                           Condicion\nHora                                \n11              Patchy rain possible\n12              Patchy rain possible\n14              Patchy rain possible\n16    Patchy light rain with thunder\n17    Patchy light rain with thunder'

In [135]:
# NUMERO
PHONE_NUMBER

'+1 720 583 9694'

In [141]:
#import os
#from twilio.rest import Client


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

client = Client(account_sid, auth_token)

message = client.messages \
    .create(
        body=mensaje,
        from_=PHONE_NUMBER,
        to='+543886857117'
    )

print('Mensaje enviado! ',message.sid)

Mensaje enviado!  SM3218edb3daf1e564ffa15d6b311ef961
